In [ ]:
!pip install razdel

In [ ]:
import json

import pandas as pd
from transformers import AutoModelForPreTraining, BertTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH_PREFIX = 'drive/MyDrive/diploma/data/'

In [ ]:
with open(DATA_PATH_PREFIX + 'e-mordovia/aligned_news_texts_06_02.json', 'r') as f:
    e_mordovia_texts_df = pd.DataFrame(json.load(f))

In [ ]:
e_mordovia_texts_df.head()

,mdf,ru
0,Россиянь Минэкономразвитиясь и «Диалог Регионы...,Минэкономразвития России и АНО «Диалог Регионы...
1,19 августста Саранскяйса панжевсь школаса шаба...,19 августа в Саранске открылся первый Всеросси...
2,Российскяй наукань шиня чествондафтольхть асу ...,В День российской науки символично отмечать та...
3,Мордовиянь Оцюнясь Артём Здунов 2 июльста вась...,Глава Мордовии Артём Здунов 2 июля встретился ...
4,16 февральста Мордовиянь Оцюнясь Артём Здунов ...,16 февраля в Москве на рабочей встрече Главы М...


# align and save

In [ ]:
model_path = 'drive/MyDrive/diploma/labse_moksha_40k+70k+2k_ce_0602_v2'

In [ ]:
model = AutoModelForPreTraining.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

In [ ]:
model.cuda();

In [ ]:
all_aligned_pairs = []

for idx, (mdf_text, ru_text) in enumerate(zip(
    e_mordovia_texts_df['mdf'],
    e_mordovia_texts_df['ru']
)):
    if not ru_text or not mdf_text:
        print(f"empty pair: ({idx}), {mdf_text}, {ru_text}")
        continue

    aligned_pairs = align_sentences(mdf_text, ru_text, model, tokenizer, threshold=0.4)
    all_aligned_pairs += aligned_pairs

    # if not aligned_pairs:
    #     print(f"0 aligned pairs: {idx}, {mdf_text}, {ru_text}")

0 aligned pairs: 172, В пятницу, 13 апреля Глава Мордовии Владимир Волков принял участие в торжественном собрании, посвященном 100-летию создания системы военных комиссариатов России и Мордовии. В Национальном драматическом театре собрались работники и ветераны военкоматов республики.
Военные комиссариаты сыграли важнейшую роль в годы Великой Отечественной войны. Благодаря самоотдаче их работников удалось оперативно провести мобилизацию, обеспечить войска техникой и ресурсами, в рекордные сроки сформировать армию, которая встала на защиту Родины. Наряду с подвигами фронтовиков и тружеников тыла это стало одним из решающих факторов в достижении Великой Победы.
«Ваша служба – одно из важнейших звеньев обороноспособности государства. На протяжении всех лет военные комиссариаты
органично слиты с историей развития нашей армии,
четко выполняют колоссальные по объемам и сложности задачи комплектования Вооруженных Сил», - обратился к собравшимся Владимир Волков.
Как отметил руководитель респуб

In [ ]:
data = []
for mdf, ru in all_aligned_pairs:
    if not is_text_valid(mdf) or not is_text_valid(ru):
        continue

    data.append({'mdf': mdf, 'ru': ru})

In [ ]:
with open(DATA_PATH_PREFIX + 'aligned_news_sents_06_02.json', "w") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)